In [1]:
from datetime import datetime

import qnexus as qnx

from pytket import Circuit

from nexus_dataclasses.backend_config import QuantinuumConfig

In [9]:
my_project = qnx.project.get_only(name="VanyaTest")

qnx.context.set_active_project(my_project)

my_job_name_prefix = datetime.now()

## Compile Jobs

In [5]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()

my_circuit_ref = qnx.circuit.upload(
    circuit=c,
    name="A circuit",
)

In [10]:
compile_job_ref = qnx.compile(
    circuits=[my_circuit_ref],
    name=f"{my_job_name_prefix}_compile",
    optimisation_level=1,
    target=QuantinuumConfig(device_name="H1-1SC")
)

In [11]:
qnx.job.wait_for(compile_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 4, 29, 15, 50, 2, 6317, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 4, 29, 15, 49, 55, 209308, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [12]:
compile_results = qnx.job.compile.results(compile_job_ref)

compile_results.df()

,name,description,project,id
0,A circuit-compilation,,VanyaTest,3a05300b-569a-4df7-bdab-aab018c7f428


In [13]:
compile_results[0].get_compiled_circuit()

CircuitRef(annotations=Annotations(name='A circuit-QuantinuumBackend-final', description=None, properties=OrderedDict()), project=ProjectRef(annotations=Annotations(name='VanyaTest', description='', properties=OrderedDict()), id=UUID('6ed750a5-de0c-4879-a771-2c1f3f746d18')), id=UUID('20162f9d-4d55-4195-9111-615d25336f3e'))

In [14]:
compile_results[0].get_compilation_passes()

[('DecomposeBoxes',
  CircuitRef(annotations=Annotations(name='A circuit', description=None, properties=OrderedDict()), project=ProjectRef(annotations=Annotations(name='VanyaTest', description='', properties=OrderedDict()), id=UUID('6ed750a5-de0c-4879-a771-2c1f3f746d18')), id=UUID('d48fa20b-5bb8-4604-894b-0f900af887ee'))),
 ('CustomPass',
  CircuitRef(annotations=Annotations(name='A circuit', description=None, properties=OrderedDict()), project=ProjectRef(annotations=Annotations(name='VanyaTest', description='', properties=OrderedDict()), id=UUID('6ed750a5-de0c-4879-a771-2c1f3f746d18')), id=UUID('d48fa20b-5bb8-4604-894b-0f900af887ee'))),
 ('SynthesiseTK',
  CircuitRef(annotations=Annotations(name='A circuit-QuantinuumBackend-2', description=None, properties=OrderedDict()), project=ProjectRef(annotations=Annotations(name='VanyaTest', description='', properties=OrderedDict()), id=UUID('6ed750a5-de0c-4879-a771-2c1f3f746d18')), id=UUID('193304cf-ca28-4db2-959f-f6d6cb73100c'))),
 ('Normalis

## Execute Jobs

In [15]:
compiled_circuits = [item.get_compiled_circuit() for item in compile_results]

In [17]:
execute_job_ref = qnx.execute(
    circuits=compiled_circuits,
    name=f"{my_job_name_prefix}1_execute",
    n_shots=[100]* len(compiled_circuits),
    target=QuantinuumConfig(device_name="H1-1SC")
)

In [18]:
qnx.job.wait_for(execute_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 4, 29, 15, 50, 35, 158883, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 4, 29, 15, 50, 22, 303721, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [19]:
execute_results = qnx.job.execute.results(execute_job_ref)

execute_results.df()

AttributeError: 'ExecutionResultRef' object has no attribute 'backend_result'

In [28]:
execute_results[0].get_backend_info()

BackendInfo(name='QuantinuumBackend', device_name='H1-1LE', version='0.32.0', architecture=<tket::FullyConnected, nodes=20>, gate_set={<OpType.Reset: 65>, <OpType.Rz: 36>, <OpType.PhasedX: 68>, <OpType.ZZMax: 70>, <OpType.ClassicalExpBox: 103>, <OpType.Barrier: 8>, <OpType.ZZPhase: 73>, <OpType.TK2: 41>, <OpType.WASM: 14>, <OpType.SetBits: 15>, <OpType.CopyBits: 16>, <OpType.RangePredicate: 17>, <OpType.ExplicitPredicate: 18>, <OpType.ExplicitModifier: 19>, <OpType.MultiBit: 20>, <OpType.Measure: 63>}, n_cl_reg=0, supports_fast_feedforward=True, supports_reset=True, supports_midcircuit_measurement=True, all_node_gate_errors=None, all_edge_gate_errors=None, all_readout_errors=None, averaged_node_gate_errors=None, averaged_edge_gate_errors=None, averaged_readout_errors=None, misc={'batching': False, 'cl_reg_width': 32, 'n_shots': 10000, 'options': {}, 'syntax_checker': 'H1-1SC', 'system_type': 'local_emulator', 'wasm': True})

In [34]:
execute_results[0].get_backend_result()

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=None,shots=[[  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]],state=None,unitary=None,density_matrix=None)

In [ ]:
# Example of a job erroring

In [35]:
execute_job_ref = qnx.execute(
    circuits=[my_circuit_ref],
    name=f"error_execute_{datetime.now()}",
    n_shots=[100],
    target=QuantinuumConfig(device_name="H1-1LE")
)

In [36]:
qnx.job.wait_for(execute_job_ref)

JobError: Job errored with detail: <class 'pytket.backends.backend_exceptions.CircuitNotValidError'>: Circuit with index 0 in submitted does not satisfy GateSetPredicate:{ ExplicitPredicate CopyBits WASM TK2 ClassicalExpBox ZZMax ZZPhase Barrier MultiBit PhasedX ExplicitModifier Rz RangePredicate Reset SetBits Measure } (try compiling with backend.get_compiled_circuits first).